# Network models
Hermina Petric Maretic, *PhD student*, [EPFL](http://epfl.ch) [LTS4](http://lts4.epfl.ch)

For this assignment we will work on a network representing the collaboration between scientists in the field of General Relativity and Quantum Cosmology. You can download this network from http://snap.stanford.edu/data/ca-GrQc.html.

In [ ]:
%matplotlib inline

import random
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
G=nx.read_edgelist("ca-GrQc.txt")

In [ ]:
print('My network has {} nodes.'.format(len(G.nodes())))
print('My network has {} edges.'.format(G.size()))

## 1. Comparing to model networks

1.1 In this part of the assignment, you have to create an Erdos-Renyi and Barabasi-Albert graph using NetworkX, and compare them to the colaboration network. Try to simulate the original network as best you can, so when choosing parameters for the networks, take into account the number of vertices and edges of the original network. Comment on your choice of parameters.

In [ ]:
n = len(G.nodes())
m = G.size()
p = m*2/(n*(n-1))
er=nx.erdos_renyi_graph(n,p)
ba=nx.barabasi_albert_graph(n,3)

For the ER network, the expected number of edges is $E[m] = pn(n-1)/2$.
For the BA network, m is fixed to $m = q*(n-q)$, where q is the model parameter. Since $q$ had to be a natural number, $q=3$ gives the best approximation.

In [ ]:
print('My Erdos Renyi network has {} nodes.'.format(len(er.nodes())))
print('My Erdos Renyi network has {} edges.'.format(er.size()))
print('My Barabasi Albert network has {} nodes.'.format(len(ba.nodes())))
print('My Barabasi Albert network has {} edges.'.format(ba.size()))

1.2 Check the size of the largest connected component in each graph and compare them to the original network. In Erdos Renyi model, what would need to be the probability of creating each edge in order to have the same expected size of the largest component? Explain. Try generating a graph with this parameter to check if you get a similar value.

In [ ]:
giant_G = max(nx.connected_component_subgraphs(G), key=len)
giant_er = max(nx.connected_component_subgraphs(er), key=len)
giant_ba = max(nx.connected_component_subgraphs(ba), key=len)
print(len(giant_G.nodes()))
print(len(giant_er.nodes()))
print(len(giant_ba.nodes()))

We know from the lectures that in Erdős–Rényi networks, the fraction of nodes in the giant component $S = \frac{N_G}{N}$ grows with the average degree by $S = 1 - e^{-\langle k \rangle S}$. Therefore, as $\langle k \rangle = (N-1)p$:

In [249]:
S_G = len(giant_G.nodes())/n
p_giant = - np.log(1-S_G)/((n-1)*S_G)
print('The parameter p for an Erdos Renyi network with the same expected size of the giant component is {}.'.format(p_giant))

er_giant = max(nx.connected_component_subgraphs(nx.erdos_renyi_graph(n,p_giant)), key=len)
print('The size of the component in a randomly generated network with this parameter is {}.'.format(len(er_giant.nodes())))

The parameter p for an Erdos Renyi network with the same expected size of the giant component is 0.00037911157202514926.
The size of the component in a randomly generated network is 4127.


1.3 Look at the clustering coefficient of the original network. Is there a network model we talked about that could have a clustering coefficient that is close? Justify.

In [ ]:
nx.average_clustering(G)

Yes, the Watts - Strogatz model has a very high clustering coefficient at the start, which can be decreased with a higher rewiring parameter.

## 2. Creating a network with a predefined degree distribution

In this part of the assignment, you will have to create a random network from a predefined degree distribution. There are several network models which can create a random network with the exact same degree distribution as the original, or with the same expected distribution as the original. For more information, you can check in section 4.8 of [the Barabasi book.](http://networksciencebook.com)

One of the most famous ones is the configuration model. The model for a graph with $L$ edges in total is constructed in the following steps:

- Assign a degree to each node, represented as stubs (half-links). The degree sequence is either generated analytically from a preselected distribution, or it is extracted from the adjacency matrix of a real network. We must start from an even number of stubs, otherwise we are left with unpaired stubs.
- Randomly select a stub pair and connect them. Then randomly choose another pair from the remaining $2L - 2$ stubs and connect them. This procedure is repeated until all stubs are paired up.

*Reminder:* A stub is a half-link, representing half of one edge. It containes one node and can be paired up with another stub to create an edge (between the two corresponding nodes).


2.1 However, this model allows for the creation of multi-links (multiple edges between the same pair of vertices) and self-loops, thus leading to a non-simple graph. In this assignment, you will implement a greedy configuration model, to avoid these problems.

The algorithm is as follows:
- Extract the degree sequence from our collaboration network.
- Assign a target degree to each node, represented as stubs or half-links. Use the degree sequence extracted from the collaboration network.
- Sort the nodes by degree. 
    - Pick the node with the highest target degree. Delete all its stubs from the list of stubs to make sure we don't create a self loop.
    - Until all its weighted degree equals its target degree: 
        - Randomly select one stub from the list of stubs (corresponding to one of the other nodes), and connect these two nodes. In case the two chosen nodes are already connected, simply increase the weight of this edge by one. Be careful to randomly select from stubs and not from nodes, as this means the chances of selecting a node will be proportional to its target degree.
    - When the number of edges adjacent to this node corresponds its target degree, go on to the second node in the list. 
    - Repeate this procedure until all stubs are paired up, or there is only one node left with a pair number of stubs. In that case, don't create a self-loop, but discard the stubs.

*Hint:* 
 - Use nx.empty_graph() to create an empty graph
 - Use `G.add_edge(node1,node2,weight = 1)` to add edges to a weighted graph and `G.edge[node1][node2]['weight'] += 1`

In [ ]:
def greedy_configuration(degree_distribution):
    N=len(degree_distribution)
    G=nx.empty_graph()

    degree_distribution = sorted(degree_distribution,reverse=True)
    stublist=[]
    for node in range(len(degree_distribution)):
        for i in range(degree_distribution[node]):
            stublist.append(node) 
    
    for node in range(len(degree_distribution)):
        for stub in range(degree_distribution[node]):
            del stublist[0]
        while(degree_distribution[node] and len(stublist) > 0):
            ind = np.random.randint(0,len(stublist))
            b = stublist.pop(ind)
            if G.has_edge(node,b):
                G.edge[node][b]['weight'] += 1
            else:
                G.add_edge(node,b,weight = 1)
            
            degree_distribution[node] = degree_distribution[node] - 1
            degree_distribution[b] = degree_distribution[b] - 1
    return G

In [ ]:
degree_sequence=sorted(nx.degree(G).values(),reverse=True) # degree sequence
gc = greedy_configuration(degree_sequence)

2.2 Check that the network has the same size and plot the differences between weighted degree distributions to check if they are same.

In [ ]:
print('My network has {} nodes.'.format(len(gc.nodes())))
degree_sequence_gc=sorted(nx.degree(gc, weight = 'weight').values(),reverse=True)
plt.plot(np.array(degree_sequence) - np.array(degree_sequence_gc));

2.3 Draw both this and the original network. Should these two networks have the same adjacency matrices? Does a node permutation change anything?

In [ ]:
nx.draw(G)

In [ ]:
nx.draw(gc)

They won't have the same adjacency matrix, because the second only shares the same degree distribution, but not the same structure.

2.4 Do you expect the properties from the first part of the assignment to stay close to the original graph? Explain.

No, as this graph is still random and has no real structure.